<a href="https://colab.research.google.com/github/elgonio/TechAwayThursdayDemo/blob/main/TechAwayDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First we install the libraries we're using


In [9]:
!pip install tweepy
!pip install nltk
!pip install sklearn

#Next we download the data we'll be training on

In [10]:
import urllib.request

file_downloader = urllib.request.URLopener()
file_downloader.retrieve("https://raw.githubusercontent.com/elgonio/TechAwayThursdayDemo/main/negative_tweets", "rt.neg")
file_downloader.retrieve("https://raw.githubusercontent.com/elgonio/TechAwayThursdayDemo/main/positive_tweets", "rt.pos")

('rt.pos', <http.client.HTTPMessage at 0x7f7b80d74b50>)

#Now we do the training


In [11]:
import nltk
nltk.download("punkt") # download if necessary
nltk.download('stopwords')
import pickle
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy

from nltk.corpus import stopwords


# this part of the code is shamelessly borrowed from:
# https://www.twilio.com/blog/2017/09/sentiment-analysis-python-messy-data-nltk.html
def format_sentence(sent):
    return({word: True for word in nltk.word_tokenize(sent)})

pos = []
with open("rt.pos", encoding="latin-1") as f:
    for i in f:
        if len(pos) > 4000: break
        pos.append([format_sentence(i), 'pos'])


neg = []
with open("rt.neg", encoding="latin-1") as f:
    for i in f:
        if len(neg) > 4000: break
        neg.append([format_sentence(i), 'neg'])

# next, split labeled data into the training and test data
training = pos[:int((.8)*len(pos))] + neg[:int((.8)*len(neg))]
test = pos[int((.8)*len(pos)):] + neg[int((.8)*len(neg)):]

classifier = NaiveBayesClassifier.train(training)
# We can show the most informative features to gain more insight into our data
# classifier.show_most_informative_features()
print("accuracy = " + str(accuracy(classifier, test)))

# training a model every time we run the program is inefiicient
# we use pickle to store and save the model for future use
classifier_file = open("classifier.pickle", "wb")
pickle.dump(classifier, classifier_file)
classifier_file.close()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
accuracy = 0.7053682896379525


In [18]:
import tweepy

# plz don't steal my twitter credentials
ckey = "nYKSjSpE5IrO9y7qECVk4jMMu"
csecret = "YdCeEYJSGHVscthswLnNiDHmM7N24hSM82hslTVsRpgmn9evD3"
atoken = "282101457-b7VzdJJnzNYzHrS6VwC7OCOzHOIKjedYI9NBP3ts"
asecret = "MRSCxDqpIxdkGHj6SDAdh693tYxkTyyKPFS9TaKBI8fHq"

OAUTH_KEYS = {'consumer_key':ckey, 'consumer_secret':csecret,
'access_token_key':atoken, 'access_token_secret':asecret}
auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])
api =tweepy.API(auth)

query = input("What do you want to search for? \n")
print('Searching...')
tweets = tweepy.Cursor(api.search, q=query, lang='en', rpp='100').items(100)
tweet_list = []
for tweet in tweets:
   # we append a new line for when we save to a file
   tweet_list.append(tweet.text.strip() + "\n")

f=open('tweet_data.txt','w', encoding="utf-8")
f.writelines(tweet_list)
print("fetched " + str(len(tweet_list)) + " tweets")

What do you want to search for? 
Eskom
Searching...
fetched 100 tweets


# Now we've fetched the tweets, let's run them through our classifier and see how people feel about our search

In [19]:
classifier_file = open("classifier.pickle", "rb")
classifier = pickle.load(classifier_file)
classifier_file.close()

score = 0
for tweet in tweet_list:
    tweet.rstrip()
    tweet = tweet.replace("\r", "")
    tweet = tweet.replace("\n", "")
    # print(tweet + " : " + classifier.classify(format_sentence(tweet)))
    if classifier.classify(format_sentence(tweet)) == "pos":
        score = score + 1
print("num tweets is: ", len(tweet_list))
print("num pos tweets is: ", score)
print("num neg tweets is: ", len(tweet_list)-score)
print("score is : ", (score/len(tweet_list))*100, "% positive")
if (score/len(tweet_list) > 0.8):
    label = "Very positive"
elif (score/len(tweet_list) > 0.6):
    label = "positive"
elif (score/len(tweet_list) > 0.4):
    label = "Neutral/Mixed"
elif (score/len(tweet_list) > 0.2):
    label = "Negative"
elif (score/len(tweet_list) > 0):
    label = "Very Negative"

print("general sentiment is %s" % label)

num tweets is:  100
num pos tweets is:  21
num neg tweets is:  79
score is :  21.0 % positive
general sentiment is Negative


# scikit-learn example


In [14]:
import numpy as np

In [15]:
cats = [[np.random.randint(5), np.random.randint(2), np.random.randint(7)] for i in range(1000)]
labels = [0 if cats[i][1] == False else 1 for i in range(len(cats))]

In [16]:
from sklearn.neural_network import MLPClassifier
X = cats
Y = labels
classifier = MLPClassifier()
classifier.fit(X,Y)

classifier.score(X,Y)

1.0